In [ ]:
# !unzip sentiment_data.zip

In [ ]:
#importing necessery libraries for future analysis of the dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
np.random.seed(2020)
import nltk
nltk.download('punkt') # one time execution
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
import re
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# from  nltk.stem import SnowballStemmer
# stemmer = SnowballStemmer("english")

In [ ]:
data_head = ["target", "ids", "date", "flag", "user", "text"]
data = pd.read_csv("sentiment_data.csv", encoding = "ISO-8859-1", names = data_head)
data.head(3)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [ ]:
data.shape

(1600000, 6)

In [ ]:
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+|\S*@\S*\s?|[^\w\s]"

def preprocess(text):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            tokens.append(token)
    return " ".join(tokens)

data.text = data.text.apply(preprocess)

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
def vectorize(value):
    sentences = value.to_list()

    sentence_vectors = []
    for i in sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)

    sentence_vectors = np.array(sentence_vectors)
    
    return sentence_vectors

In [ ]:
text_vec = vectorize(data.text)
np.save("sentiment_input.npy", text_vec)
print(text_vec.shape)

(1600000, 100)


In [ ]:
target = np.array(data.target.to_list())
target = target.reshape(-1, 1)
np.save("target.npy", target)
print(target.shape)

(1600000, 1)
